In [ ]:
#Importing dependencies
import pandas as pd
import numpy as np
import sqlite3
from datetime import date

In [ ]:
# Import SQLAlchemy `automap` and other dependencies here
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, inspect, func
import psycopg2 
import io

engine = create_engine('sqlite:///database/database.sqlite')
inspector = inspect(engine)
inspector.get_table_names()

In [ ]:
# Use `engine.execute` to select and display the first 10 rows from the player table
engine.execute('SELECT * FROM Player LIMIT 10').fetchall()

In [ ]:
# Reflect Database into ORM classes
Base = automap_base()
Base.prepare(engine, reflect=True)
Player = Base.classes.Player
League = Base.classes.League
Match = Base.classes.Match
Player_Attributes = Base.classes.Player_Attributes
Team = Base.classes.Team
Country = Base.classes.Country
Team_Attributes = Base.classes.Team_Attributes

In [ ]:
# Use Pandas `read_sql_query` to load a query statement directly into the DataFrame
# Start a session to query the database
session = Session(engine)
player = session.query(Player).statement
League = session.query(League).statement
Match = session.query(Match).statement
Player_Stats = session.query(Player_Attributes).statement
Country = session.query(Country).statement
Team = session.query(Team).statement
player_df = pd.read_sql_query(player, session.bind)
player_df.head(10)

In [ ]:
session.close()

In [ ]:
match_df = pd.read_sql_query(Match, session.bind)
match_df.head()

In [ ]:
player_stats_df = pd.read_sql_query(Player_Stats, session.bind)
player_stats_df

In [ ]:
#Joining Palyer table with stats
df = pd.merge(player_df, player_stats_df, on='player_api_id', how='left', suffixes=('', '_stats')).drop_duplicates(subset='player_api_id')
df

In [ ]:
df['age'] = np.nan
for i, value in df['date'].items():
    df.loc[i,'age'] = (pd.to_datetime(df.loc[i,'date'].split(' ')[0]).year - pd.to_datetime(df.loc[i,'birthday'].split(' ')[0]).year)
        
df.head()

In [ ]:
group_age = df.groupby('age').mean().round(2).reset_index()
group_age 

In [ ]:
age_groups = [17, 20,25,30, 35,40, 44]
age_labels =['<20','20-25', '26-30','31-35','36-40','>40']
group_age['Age_Groups'] = pd.cut(group_age.index, age_groups, labels=age_labels)
Age_cat = group_age.groupby('Age_Groups').mean().round(2)
Age_cat

In [ ]:
match_df = pd.read_sql_query(Match, session.bind)
match_df.head()

In [ ]:
team_df = pd.read_sql_query(Team, session.bind)
team_df

In [ ]:
#Joining team table with the match
match_team_df = pd.merge(match_df, team_df, left_on='home_team_api_id', right_on = 'team_api_id',how='left', suffixes=('_match', '_team'), validate = 'm:1')
match_team_df.rename(columns={'team_long_name':'Home_Team'}, inplace=True)
match_team_df = pd.merge(match_team_df, team_df, left_on='away_team_api_id', right_on = 'team_api_id',how='left', suffixes=('_match', '_team'), validate = 'm:1')
match_team_df.rename(columns={'team_long_name':'away_Team'}, inplace=True)
columns_of_interest = ['id_match','country_id','league_id','season','stage','date','match_api_id','Home_Team','away_Team','home_team_goal',
'away_team_goal']
match_team_df = match_team_df[columns_of_interest]
match_team_df = pd.merge(match_team_df, league_df, left_on='league_id', right_on = 'id',how='left', suffixes=('_match', '_team'), validate = 'm:1')
match_team_df.drop(['id','league_id'], axis=1)
match_team_df.rename(columns={'name':'League'}, inplace=True)
match_team_df = match_team_df[['country_id_match','League','season','stage','date','match_api_id','Home_Team','away_Team','home_team_goal',
'away_team_goal']]
match_team_df = pd.merge(match_team_df, country_df, left_on='country_id_match', right_on = 'id',how='left', suffixes=('_match', '_team'), validate = 'm:1')
match_team_df.rename(columns={'name':'Country'}, inplace=True)
match_team_df.drop(['country_id_match', 'match_api_id', 'id', 'stage'], axis=1, inplace=True)
match_team_df

In [ ]:
match_team_df

In [ ]:
match_team_df.groupby(['Home_Team','away_Team']).sum()

In [ ]:
league_df = pd.read_sql_query(League, session.bind)
league_df

In [ ]:
country_df = pd.read_sql_query(Country, session.bind)
country_df

In [ ]:
league_country = pd.merge(league_df, country_df, on ='id', suffixes=('_league', '_country'))
league_country

In [ ]:
#Adding two columns, Score, Winner.
match_team_df['score_difference'] = match_team_df['home_team_goal'] - match_team_df['away_team_goal']
match_team_df['winner'] = np.nan
for i, v in match_team_df['score_difference'].items():
    if v > 0:
        match_team_df.loc[i,'winner'] = match_team_df.loc[i,'Home_Team']
    elif v<0:
        match_team_df.loc[i,'winner'] = match_team_df.loc[i,'away_Team']
    else:
        match_team_df.loc[i,'winner'] = 'Null_Match'
match_team_df.drop(['score'], axis=1, inplace=True)
match_team_df  

In [ ]:
span_league = match_team_df[match_team_df['Country'] == 'Spain']
span_league_scores = span_league.groupby("winner").sum().sort_values("score_difference", ascending = False).reset_index()
span_league_scores
# Barcelona 💪

In [ ]:
engine2 = create_engine('postgresql://postgres:@localhost:5432/fifa_db')

In [ ]:
inspector = inspect(engine2)
inspector.get_table_names()

In [ ]:
session = Session(engine2)

In [ ]:
#span_league_scores.to_sql('spain_league', engine2)
span_league_scores.to_sql(name='spain_league', con=engine2, if_exists = 'replace', index=False)
match_team_df.to_sql(name='match', con=engine2, if_exists = 'replace', index = False)
group_age.to_sql(name='Age_group', con=engine2, if_exists = 'replace', index = False)
player_df.to_sql(name='Player', con=engine2, if_exists = 'replace', index = False)

In [ ]:
session.close